# ProductionScore 구하는 코드
- 필요 코드 : maindata_var1.2.2.csv, movies_metadata_date.csv
- 아웃풋 코드 : movies_cleaned_removed_onehot_director2.csv

In [1]:
import pandas as pd
import numpy as np
import ast

In [2]:
maindata = pd.read_csv('data/maindata_var1.2.2.csv')

maindata.head()

,director_name,director_id,id,budget,popularity,revenue,runtime,title,vote_average,vote_count,...,actor2_id,actor2_name,actor3_id,actor3_name,actor1_score,actor2_score,actor3_score,actor_score,company_name,company_id
0,John Lasseter,7879.0,862,30000000,21.946943,394436586,81.0,Toy Story,7.7,5415.0,...,12898.0,Tim Allen,7167.0,Don Rickles,2.314125e+08,1.898334e+08,0.000000e+00,210622908,['Pixar Animation Studios'],[3]
1,Howard Deutch,26502.0,15602,25000000,11.712900,71500000,101.0,Grumpier Old Men,6.5,92.0,...,3151.0,Jack Lemmon,13567.0,Ann-Margret,1.048144e+08,6.282956e+07,3.645318e+07,68032369,"['Warner Bros.', 'Lancaster Gate']","[6194, 19464]"
2,Martin Campbell,10702.0,710,58000000,14.686036,352200000,130.0,GoldenEye,6.6,1194.0,...,48.0,Sean Bean,10695.0,Izabella Scorupco,1.638866e+08,1.780516e+08,0.000000e+00,170969111,"['United Artists', 'Eon Productions']","[60, 7576]"
3,Steve Oedekerk,4489.0,9273,30000000,8.205448,212385533,90.0,Ace Ventura: When Nature Calls,6.1,1128.0,...,3547.0,Ian McNeice,4001.0,Simon Callow,2.085212e+08,1.916900e+07,9.343130e+07,107040513,"['O Entertainment', 'Warner Bros.', 'Morgan Cr...","[5682, 6194, 10210]"
4,Barry Sonnenfeld,5174.0,8012,30250000,12.669608,115101622,105.0,Get Shorty,6.4,305.0,...,193.0,Gene Hackman,14343.0,Rene Russo,6.889426e+07,9.699749e+07,1.764293e+08,114107021,"['Jersey Films', 'Metro-Goldwyn-Mayer (MGM)']","[216, 8411]"


In [3]:
metadata = pd.read_csv('data/movies_metadata.csv')

/var/folders/_4/mvxsg_zj2q35755ky_zps0780000gn/T/ipykernel_19728/2984535404.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata = pd.read_csv('data/movies_metadata.csv')


In [4]:
# Convert release_date to datetime
metadata['release_date'] = pd.to_datetime(metadata['release_date'], errors='coerce')

In [5]:
# Step 2: Extract production company data and create a dictionary for movies produced
# Handle NaN values by replacing them with an empty list before applying ast.literal_eval
metadata['production_companies'] = metadata['production_companies'].fillna('[]').apply(ast.literal_eval)

# Create a dictionary to hold movie information for each company
company_movies_dict = {}

for idx, row in metadata.iterrows():
    release_date = row['release_date']
    movie_id = row['id']
    
    if pd.isna(release_date) or pd.isna(movie_id):
        continue

    production_companies = row['production_companies']
    for company in production_companies:
        if isinstance(company, dict) and 'id' in company:
            company_id = company['id']

            # Update the company_movies_dict
            if company_id not in company_movies_dict:
                company_movies_dict[company_id] = []
            company_movies_dict[company_id].append((release_date, movie_id))

In [6]:
maindata

,director_name,director_id,id,budget,popularity,revenue,runtime,title,vote_average,vote_count,...,actor2_id,actor2_name,actor3_id,actor3_name,actor1_score,actor2_score,actor3_score,actor_score,company_name,company_id
0,John Lasseter,7879.0,862,30000000,21.946943,394436586,81.0,Toy Story,7.7,5415.0,...,12898.0,Tim Allen,7167.0,Don Rickles,2.314125e+08,1.898334e+08,0.000000e+00,210622908,['Pixar Animation Studios'],[3]
1,Howard Deutch,26502.0,15602,25000000,11.712900,71500000,101.0,Grumpier Old Men,6.5,92.0,...,3151.0,Jack Lemmon,13567.0,Ann-Margret,1.048144e+08,6.282956e+07,3.645318e+07,68032369,"['Warner Bros.', 'Lancaster Gate']","[6194, 19464]"
2,Martin Campbell,10702.0,710,58000000,14.686036,352200000,130.0,GoldenEye,6.6,1194.0,...,48.0,Sean Bean,10695.0,Izabella Scorupco,1.638866e+08,1.780516e+08,0.000000e+00,170969111,"['United Artists', 'Eon Productions']","[60, 7576]"
3,Steve Oedekerk,4489.0,9273,30000000,8.205448,212385533,90.0,Ace Ventura: When Nature Calls,6.1,1128.0,...,3547.0,Ian McNeice,4001.0,Simon Callow,2.085212e+08,1.916900e+07,9.343130e+07,107040513,"['O Entertainment', 'Warner Bros.', 'Morgan Cr...","[5682, 6194, 10210]"
4,Barry Sonnenfeld,5174.0,8012,30250000,12.669608,115101622,105.0,Get Shorty,6.4,305.0,...,193.0,Gene Hackman,14343.0,Rene Russo,6.889426e+07,9.699749e+07,1.764293e+08,114107021,"['Jersey Films', 'Metro-Goldwyn-Mayer (MGM)']","[216, 8411]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1235,Matt Reeves,32278.0,281338,150000000,146.161786,490719763,140.0,War for the Planet of the Apes,6.7,1675.0,...,57755.0,Woody Harrelson,18324.0,Steve Zahn,1.301344e+09,2.625799e+08,1.123724e+08,558765400,['Chernin Entertainment'],[7076]
1236,Roman Prygunov,235774.0,327982,2265160,1.369751,9194350,0.0,Soulless 2,4.4,11.0,...,2413.0,Dominique Pinon,NaN,NaN,1.255907e+07,8.511010e+06,0.000000e+00,10535042,"['Art Pictures Studio', 'Kinoslovo']","[3451, 21179]"
1237,Michael Bay,865.0,335988,217000000,39.186819,605425157,149.0,Transformers: The Last Knight,6.2,1440.0,...,19536.0,Josh Duhamel,209578.0,Laura Haddock,2.313437e+08,2.318618e+07,8.182663e+08,357598724,"['Paramount Pictures', 'Di Bonaventura Picture...","[4, 435, 12087, 22826]"
1238,Timur Bekmambetov,23430.0,313556,2426950,1.361724,12848123,0.0,Six Degrees of Celebration 1914,3.8,4.0,...,122539.0,Ivan Urgant,143623.0,Aleksandr Golovin,2.362124e+07,2.503026e+07,2.503026e+07,24560590,['Bazelevs Production'],[1038]


In [7]:
company_movies_dict

{3: [(Timestamp('1995-10-30 00:00:00'), '862'),
  (Timestamp('1998-11-25 00:00:00'), '9487'),
  (Timestamp('1999-10-30 00:00:00'), '863'),
  (Timestamp('2001-11-01 00:00:00'), '585'),
  (Timestamp('2003-05-30 00:00:00'), '12'),
  (Timestamp('2004-11-05 00:00:00'), '9806'),
  (Timestamp('1986-08-17 00:00:00'), '13925'),
  (Timestamp('2006-06-08 00:00:00'), '920'),
  (Timestamp('2007-06-22 00:00:00'), '2062'),
  (Timestamp('2008-06-22 00:00:00'), '10681'),
  (Timestamp('2009-05-13 00:00:00'), '14160'),
  (Timestamp('2009-05-28 00:00:00'), '24480'),
  (Timestamp('2010-06-16 00:00:00'), '10193'),
  (Timestamp('2010-06-17 00:00:00'), '40619'),
  (Timestamp('2008-11-17 00:00:00'), '13413'),
  (Timestamp('2011-06-11 00:00:00'), '49013'),
  (Timestamp('2002-05-24 00:00:00'), '13931'),
  (Timestamp('2006-12-28 00:00:00'), '13060'),
  (Timestamp('2012-06-21 00:00:00'), '62177'),
  (Timestamp('2008-06-18 00:00:00'), '13042'),
  (Timestamp('2004-12-31 00:00:00'), '13932'),
  (Timestamp('2003-11-04

In [8]:
# Step 3: Iterate over maindata to calculate recent 5-year movie lists for each production company
recent_7_year_movie_ids = []

for idx, row in maindata.iterrows():
    # release_year, release_month, release_day를 사용하여 release_date 생성
    try:
        # 연도, 월, 일을 문자열로 합쳐서 하나의 날짜 형식으로 만듦 (예: "2017-03-02")
        release_date_str = f"{row['release_year']}-{row['release_month']:02d}-{row['release_day']:02d}"
        release_date = pd.to_datetime(release_date_str, errors='coerce')
    except:
        release_date = pd.NaT

    # 제작사 정보 추출
    production_companies = ast.literal_eval(row['company_id']) if pd.notna(row['company_id']) else []

    # 각 제작사별로 최근 7년간 제작한 영화 ID 리스트 구하기
    company_movie_lists = []

    for company_id in production_companies:
        # 회사별로 최근 7년 동안의 영화 ID 추출
        if company_id in company_movies_dict:
            cutoff_date = release_date - pd.DateOffset(years=7)
            movies_in_last_7_years = [
                movie_id for date, movie_id in company_movies_dict[company_id]
                if pd.notna(date) and cutoff_date < date < release_date
            ]
            company_movie_lists.append(movies_in_last_7_years)

    # 결과 추가
    recent_7_year_movie_ids.append(company_movie_lists)

# Add column to maindata
maindata['recent_7_year_movie_ids'] = recent_7_year_movie_ids

In [9]:
# company_movies_dict

In [10]:
maindata

,director_name,director_id,id,budget,popularity,revenue,runtime,title,vote_average,vote_count,...,actor2_name,actor3_id,actor3_name,actor1_score,actor2_score,actor3_score,actor_score,company_name,company_id,recent_7_year_movie_ids
0,John Lasseter,7879.0,862,30000000,21.946943,394436586,81.0,Toy Story,7.7,5415.0,...,Tim Allen,7167.0,Don Rickles,2.314125e+08,1.898334e+08,0.000000e+00,210622908,['Pixar Animation Studios'],[3],[[13928]]
1,Howard Deutch,26502.0,15602,25000000,11.712900,71500000,101.0,Grumpier Old Men,6.5,92.0,...,Jack Lemmon,13567.0,Ann-Margret,1.048144e+08,6.282956e+07,3.645318e+07,68032369,"['Warner Bros.', 'Lancaster Gate']","[6194, 19464]","[[949, 9273, 9691, 33689, 11859, 688, 27793, 3..."
2,Martin Campbell,10702.0,710,58000000,14.686036,352200000,130.0,GoldenEye,6.6,1194.0,...,Sean Bean,10695.0,Izabella Scorupco,1.638866e+08,1.780516e+08,0.000000e+00,170969111,"['United Artists', 'Eon Productions']","[60, 7576]","[[451, 11780, 10428, 8973, 10802, 380, 11497, ..."
3,Steve Oedekerk,4489.0,9273,30000000,8.205448,212385533,90.0,Ace Ventura: When Nature Calls,6.1,1128.0,...,Ian McNeice,4001.0,Simon Callow,2.085212e+08,1.916900e+07,9.343130e+07,107040513,"['O Entertainment', 'Warner Bros.', 'Morgan Cr...","[5682, 6194, 10210]","[[], [9691, 11859, 688, 27793, 30765, 414, 907..."
4,Barry Sonnenfeld,5174.0,8012,30250000,12.669608,115101622,105.0,Get Shorty,6.4,305.0,...,Gene Hackman,14343.0,Rene Russo,6.889426e+07,9.699749e+07,1.764293e+08,114107021,"['Jersey Films', 'Metro-Goldwyn-Mayer (MGM)']","[216, 8411]","[[680, 2788, 10410], [9348, 15128, 178, 18658,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1235,Matt Reeves,32278.0,281338,150000000,146.161786,490719763,140.0,War for the Planet of the Apes,6.7,1675.0,...,Woody Harrelson,18324.0,Steve Zahn,1.301344e+09,2.625799e+08,1.123724e+08,558765400,['Chernin Entertainment'],[7076],"[[61791, 88042, 75612, 136795, 119450, 154400,..."
1236,Roman Prygunov,235774.0,327982,2265160,1.369751,9194350,0.0,Soulless 2,4.4,11.0,...,Dominique Pinon,NaN,NaN,1.255907e+07,8.511010e+06,0.000000e+00,10535042,"['Art Pictures Studio', 'Kinoslovo']","[3451, 21179]","[[16911, 203793, 18122, 316269, 132759, 326011..."
1237,Michael Bay,865.0,335988,217000000,39.186819,605425157,149.0,Transformers: The Last Knight,6.2,1440.0,...,Josh Duhamel,209578.0,Laura Haddock,2.313437e+08,2.318618e+07,8.182663e+08,357598724,"['Paramount Pictures', 'Di Bonaventura Picture...","[4, 435, 12087, 22826]","[[10196, 38778, 39440, 41436, 45317, 44264, 39..."
1238,Timur Bekmambetov,23430.0,313556,2426950,1.361724,12848123,0.0,Six Degrees of Celebration 1914,3.8,4.0,...,Ivan Urgant,143623.0,Aleksandr Golovin,2.362124e+07,2.503026e+07,2.503026e+07,24560590,['Bazelevs Production'],[1038],"[[8909, 31162, 71469, 50357, 57809, 85317, 723..."


In [11]:
# Step 4: Calculate average revenue for each company in the recent 7-year movie lists
production_scores = []

for movie_lists in recent_7_year_movie_ids:
    avg_revenues = []

    # 각 제작사별로 수익 평균 계산
    for movie_ids in movie_lists:
        revenues = []
        for movie_id in movie_ids:
            # metadata에서 해당 영화의 revenue 가져오기
            if movie_id in metadata['id'].values:
                revenue = metadata.loc[metadata['id'] == movie_id, 'revenue'].values[0]
                if revenue > 0:  # revenue가 0인 경우 제외
                    revenues.append(revenue)

        # 유효한 수익이 있는 경우 평균 계산
        if revenues:
            avg_revenue = np.mean(revenues)
            avg_revenues.append(avg_revenue)

    # 각 제작사별 평균 수익 중 최대값 계산
    production_score = max(avg_revenues) if avg_revenues else 0
    # print(production_score)
    production_scores.append(production_score)

# Add production_score column to maindata
maindata['production_score'] = production_scores

In [12]:
#production_score 정수로 변환
maindata['production_score'] = maindata['production_score'].astype(int)

maindata.head()

,director_name,director_id,id,budget,popularity,revenue,runtime,title,vote_average,vote_count,...,actor3_id,actor3_name,actor1_score,actor2_score,actor3_score,actor_score,company_name,company_id,recent_7_year_movie_ids,production_score
0,John Lasseter,7879.0,862,30000000,21.946943,394436586,81.0,Toy Story,7.7,5415.0,...,7167.0,Don Rickles,2.314125e+08,1.898334e+08,0.000000e+00,210622908,['Pixar Animation Studios'],[3],[[13928]],0
1,Howard Deutch,26502.0,15602,25000000,11.712900,71500000,101.0,Grumpier Old Men,6.5,92.0,...,13567.0,Ann-Margret,1.048144e+08,6.282956e+07,3.645318e+07,68032369,"['Warner Bros.', 'Lancaster Gate']","[6194, 19464]","[[949, 9273, 9691, 33689, 11859, 688, 27793, 3...",87845993
2,Martin Campbell,10702.0,710,58000000,14.686036,352200000,130.0,GoldenEye,6.6,1194.0,...,10695.0,Izabella Scorupco,1.638866e+08,1.780516e+08,0.000000e+00,170969111,"['United Artists', 'Eon Productions']","[60, 7576]","[[451, 11780, 10428, 8973, 10802, 380, 11497, ...",156167015
3,Steve Oedekerk,4489.0,9273,30000000,8.205448,212385533,90.0,Ace Ventura: When Nature Calls,6.1,1128.0,...,4001.0,Simon Callow,2.085212e+08,1.916900e+07,9.343130e+07,107040513,"['O Entertainment', 'Warner Bros.', 'Morgan Cr...","[5682, 6194, 10210]","[[], [9691, 11859, 688, 27793, 30765, 414, 907...",83477178
4,Barry Sonnenfeld,5174.0,8012,30250000,12.669608,115101622,105.0,Get Shorty,6.4,305.0,...,14343.0,Rene Russo,6.889426e+07,9.699749e+07,1.764293e+08,114107021,"['Jersey Films', 'Metro-Goldwyn-Mayer (MGM)']","[216, 8411]","[[680, 2788, 10410], [9348, 15128, 178, 18658,...",117004306


In [13]:
company_id_lengths = maindata['company_id'].apply(lambda x: len(eval(x)))

### production_score 0인 경우 채우는 로직

- 시각화 통해서 company_id 갯수별 평균 production_score가 정비례 관계인 것 확인.
- company_id 갯수별 평균 production_score로 채워주기

In [14]:
# production_score가 0이고 company_id의 리스트 요소 갯수가 1인 경우 production_score 94325413로 변경
# production_score가 0이고 company_id 갯수가 2인 경우 production_score 139698475로 변경
# production_score가 0이고 company_id 갯수가 3인 경우 production_score 174224580로 변경
# production_score가 0이고 company_id 갯수가 4인 경우 production_score 210608101로 변경
# production_score가 0이고 company_id 갯수가 6인 경우 production_score 245379094로 변경

maindata.loc[(maindata['production_score'] == 0) & (company_id_lengths == 1), 'production_score'] = 94325413
maindata.loc[(maindata['production_score'] == 0) & (company_id_lengths == 2), 'production_score'] = 139698475
maindata.loc[(maindata['production_score'] == 0) & (company_id_lengths == 3), 'production_score'] = 174224580
maindata.loc[(maindata['production_score'] == 0) & (company_id_lengths == 4), 'production_score'] = 210608101
maindata.loc[(maindata['production_score'] == 0) & (company_id_lengths == 6), 'production_score'] = 245379094


In [15]:
maindata.head()

,director_name,director_id,id,budget,popularity,revenue,runtime,title,vote_average,vote_count,...,actor3_id,actor3_name,actor1_score,actor2_score,actor3_score,actor_score,company_name,company_id,recent_7_year_movie_ids,production_score
0,John Lasseter,7879.0,862,30000000,21.946943,394436586,81.0,Toy Story,7.7,5415.0,...,7167.0,Don Rickles,2.314125e+08,1.898334e+08,0.000000e+00,210622908,['Pixar Animation Studios'],[3],[[13928]],94325413
1,Howard Deutch,26502.0,15602,25000000,11.712900,71500000,101.0,Grumpier Old Men,6.5,92.0,...,13567.0,Ann-Margret,1.048144e+08,6.282956e+07,3.645318e+07,68032369,"['Warner Bros.', 'Lancaster Gate']","[6194, 19464]","[[949, 9273, 9691, 33689, 11859, 688, 27793, 3...",87845993
2,Martin Campbell,10702.0,710,58000000,14.686036,352200000,130.0,GoldenEye,6.6,1194.0,...,10695.0,Izabella Scorupco,1.638866e+08,1.780516e+08,0.000000e+00,170969111,"['United Artists', 'Eon Productions']","[60, 7576]","[[451, 11780, 10428, 8973, 10802, 380, 11497, ...",156167015
3,Steve Oedekerk,4489.0,9273,30000000,8.205448,212385533,90.0,Ace Ventura: When Nature Calls,6.1,1128.0,...,4001.0,Simon Callow,2.085212e+08,1.916900e+07,9.343130e+07,107040513,"['O Entertainment', 'Warner Bros.', 'Morgan Cr...","[5682, 6194, 10210]","[[], [9691, 11859, 688, 27793, 30765, 414, 907...",83477178
4,Barry Sonnenfeld,5174.0,8012,30250000,12.669608,115101622,105.0,Get Shorty,6.4,305.0,...,14343.0,Rene Russo,6.889426e+07,9.699749e+07,1.764293e+08,114107021,"['Jersey Films', 'Metro-Goldwyn-Mayer (MGM)']","[216, 8411]","[[680, 2788, 10410], [9348, 15128, 178, 18658,...",117004306


In [16]:
maindata.loc[maindata['production_score'] == 0]

,director_name,director_id,id,budget,popularity,revenue,runtime,title,vote_average,vote_count,...,actor3_id,actor3_name,actor1_score,actor2_score,actor3_score,actor_score,company_name,company_id,recent_7_year_movie_ids,production_score
400,Sally Mattison,98939.0,27767,350000,1.986075,1242995,87.0,Slumber Party Massacre III,5.4,20.0,...,97880.0,Maria Ford,0.000000e+00,0.000000e+00,0.0,0,[],[],[],0
538,Dale Schott,85365.0,19975,3400000,2.508783,12000000,76.0,Care Bears Movie II: A New Generation,5.5,11.0,...,34985.0,Cree Summer,0.000000e+00,0.000000e+00,0.0,6452690,[],[],[],0
568,Hideo Nakata,26875.0,9669,6000000,5.277814,31365000,92.0,Ringu 2,5.4,81.0,...,13253.0,Kyoko Fukada,1.603147e+07,1.940000e+07,0.0,17715737,[],[],[],0
670,Matthew Barney,116018.0,23985,1700000,0.695043,7982,79.0,Cremaster 2,5.8,9.0,...,NaN,NaN,0.000000e+00,0.000000e+00,0.0,0,[],[],[],0
674,Matthew Barney,116018.0,23986,4000000,0.806208,11047,182.0,Cremaster 3,6.8,10.0,...,NaN,NaN,0.000000e+00,0.000000e+00,0.0,0,[],[],[],0
692,Norio Tsuruta,58677.0,9674,6000000,5.007142,11918069,99.0,Ringu 0,5.8,49.0,...,27788.0,Kumiko Asou,0.000000e+00,0.000000e+00,0.0,5918692,[],[],[],0
751,Rakesh Roshan,1032103.0,32740,5000000,4.431245,16000000,154.0,Krrish,5.5,47.0,...,77234.0,Priyanka Chopra,8.533333e+06,9.513260e+07,8191541.0,37285825,[],[],[],0
825,Sanaa Hamri,64140.0,10188,27000000,4.631978,44352417,117.0,The Sisterhood of the Traveling Pants 2,6.1,244.0,...,59174.0,America Ferrera,9.889344e+07,3.909845e+07,25258803.0,54416898,[],[],[],0
975,Sajid Khan,14086.0,85052,6071922,1.047079,25097277,145.0,Housefull 2,5.0,28.0,...,84957.0,Ritesh Deshmukh,1.727839e+07,8.633333e+06,16000000.0,13970573,[],[],[],0
1003,Andrew Morahan,53599.0,19139,10000000,3.128836,7600,91.0,Goal! III : Taking On The World,3.8,36.0,...,30710.0,J. J. Feild,1.773789e+07,1.557717e+07,5874530.0,13063197,[],[],[],0


In [17]:
# 저장
# maindata.to_csv('data/maindata_ver2.0.csv', index=False)

In [18]:
# # production_score 0이 아닌 데이터 출력
# temp_df = maindata.loc[maindata['production_score'] != 0]

# # 데이터 저장
# temp_df.to_csv('data/production_tempdata.csv', index=False)